<a href="https://colab.research.google.com/github/elisaonder/master-thesis/blob/Franziska/ideological_modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import re
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

In [ ]:
df_corpus = pd.read_csv ("cmp_quasisentences_per_codes_with_names.csv")

df_corpus

,manifesto_id,party,party_name,party_abbrev,country_name,date,election_date,language,title,progtype,text,per_code
0,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,200609,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,Miljöpartiet de Gröna går till val på en polit...,per408
1,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,200609,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,"fler företag,",per402
2,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,200609,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,"levande landsbygd,",per301
3,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,200609,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,nya livschanser,per504
4,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,200609,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,och ett långsiktigt miljöansvar.,per501
...,...,...,...,...,...,...,...,...,...,...,...,...
601565,53952_202002,53952,Aontú,NaN,Ireland,202002,NaN,english,Think outside the political cartel: Vótáil Aon...,NaN,To implement a formal structure between An Com...,per601.1
601566,53952_202002,53952,Aontú,NaN,Ireland,202002,NaN,english,Think outside the political cartel: Vótáil Aon...,NaN,Standardization of Irish language exams in tea...,per601.1
601567,53952_202002,53952,Aontú,NaN,Ireland,202002,NaN,english,Think outside the political cartel: Vótáil Aon...,NaN,A unit to be established in the Department of ...,per601.1
601568,53952_202002,53952,Aontú,NaN,Ireland,202002,NaN,english,Think outside the political cartel: Vótáil Aon...,NaN,Appropriate provision of special educational n...,per601.1


In [ ]:
df_corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601570 entries, 0 to 601569
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   manifesto_id   601570 non-null  object 
 1   party          601570 non-null  int64  
 2   party_name     601570 non-null  object 
 3   party_abbrev   541145 non-null  object 
 4   country_name   601570 non-null  object 
 5   date           601570 non-null  int64  
 6   election_date  0 non-null       float64
 7   language       601570 non-null  object 
 8   title          601570 non-null  object 
 9   progtype       0 non-null       float64
 10  text           601570 non-null  object 
 11  per_code       601570 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 55.1+ MB


In [ ]:
#Changing date into year. Do not need month, so thats removed.

# Get the index position of the 'date' column
pos = df_corpus.columns.get_loc('date')

# Create a new 'year' column by extracting the first 4 digits
df_corpus.insert(pos, 'year', df_corpus['date'].astype(str).str[:4].astype(int))

# Drop the old 'date' column
df_corpus = df_corpus.drop(columns='date')

df_corpus.head()


,manifesto_id,party,party_name,party_abbrev,country_name,year,election_date,language,title,progtype,text,per_code
0,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,Miljöpartiet de Gröna går till val på en polit...,per408
1,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,"fler företag,",per402
2,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,"levande landsbygd,",per301
3,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,nya livschanser,per504
4,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,NaN,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,NaN,och ett långsiktigt miljöansvar.,per501


In [ ]:
# Remove election date and progtype since they are completely missing values.

# Drop only if the columns exist
df_corpus = df_corpus.drop(columns=["election_date", "progtype"])

df_corpus.head()

,manifesto_id,party,party_name,party_abbrev,country_name,year,language,title,text,per_code
0,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,Miljöpartiet de Gröna går till val på en polit...,per408
1,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,"fler företag,",per402
2,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,"levande landsbygd,",per301
3,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,nya livschanser,per504
4,11110_200609,11110,Miljöpartiet de Gröna,MP,Sweden,2006,swedish,Valmanifest 2006. Grönare Sverige! – för ökad ...,och ett långsiktigt miljöansvar.,per501


In [ ]:
cols = ['party_abbrev']
        #'pervote', 'voteest', 'absseat', 'corpusversion']

# Show rows with missing values in any of these columns
missing_rows = df_corpus[df_corpus[cols].isna().any(axis=1)]

# Display results
print(f"Number of rows with missing values: {len(missing_rows)}")
missing_rows

Number of rows with missing values: 60425


,manifesto_id,party,party_name,party_abbrev,country_name,year,language,title,text,per_code
26582,13001_200711,13001,Liberal Alliance,NaN,Denmark,2007,danish,Title Missing,Ny Alliance ønsker at gøre op med den blokpoli...,per305
26583,13001_200711,13001,Liberal Alliance,NaN,Denmark,2007,danish,Title Missing,Vi ønsker et opgør med Dansk Folkepartis domin...,per305
26584,13001_200711,13001,Liberal Alliance,NaN,Denmark,2007,danish,Title Missing,Vi ønsker i stedet et bredt samarbejde hen ove...,per202
26585,13001_200711,13001,Liberal Alliance,NaN,Denmark,2007,danish,Title Missing,og vi vil udvikle den danske tradition for åbe...,per202
26586,13001_200711,13001,Liberal Alliance,NaN,Denmark,2007,danish,Title Missing,Vores politik er for alle danskere.,per202
...,...,...,...,...,...,...,...,...,...,...
601565,53952_202002,53952,Aontú,NaN,Ireland,2020,english,Think outside the political cartel: Vótáil Aon...,To implement a formal structure between An Com...,per601.1
601566,53952_202002,53952,Aontú,NaN,Ireland,2020,english,Think outside the political cartel: Vótáil Aon...,Standardization of Irish language exams in tea...,per601.1
601567,53952_202002,53952,Aontú,NaN,Ireland,2020,english,Think outside the political cartel: Vótáil Aon...,A unit to be established in the Department of ...,per601.1
601568,53952_202002,53952,Aontú,NaN,Ireland,2020,english,Think outside the political cartel: Vótáil Aon...,Appropriate provision of special educational n...,per601.1


In [ ]:
# Label maps
label_list = sorted(df_corpus["per_code"].unique())
label2id = {l:i for i,l in enumerate(label_list)}
id2label = {i:l for l,i in label2id.items()}
df_corpus["label"] = df_corpus["per_code"].map(label2id)

# Grouped splits (by manifesto_id)
groups = df_corpus["manifesto_id"]
gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
train_idx, hold_idx = next(gss.split(df_corpus, groups=groups))
df_train = df_corpus.iloc[train_idx]
df_hold  = df_corpus.iloc[hold_idx]

# Split hold-out into val/test, again by manifesto_id
gss2 = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=43)
val_idx, test_idx = next(gss2.split(df_hold, groups=df_hold["manifesto_id"]))
df_val  = df_hold.iloc[val_idx]
df_test = df_hold.iloc[test_idx]

len(df_train), len(df_val), len(df_test), len(label_list)

(492547, 56145, 52878, 89)

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer

model_name = "xlm-roberta-base"  # later also try: "neuralmind/bert-base-portuguese-cased" (BERTimbau)
tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def tokenize(batch):
    return tok(batch["text"], truncation=True, padding="max_length", max_length=256)

train_ds = Dataset.from_pandas(df_train[["text","label"]])
val_ds   = Dataset.from_pandas(df_val[["text","label"]])
test_ds  = Dataset.from_pandas(df_test[["text","label"]])

train_ds = train_ds.map(tokenize, batched=True).remove_columns(["text"]).rename_column("label","labels")
val_ds   = val_ds.map(tokenize, batched=True).remove_columns(["text"]).rename_column("label","labels")
test_ds  = test_ds.map(tokenize, batched=True).remove_columns(["text"]).rename_column("label","labels")

train_ds.set_format("torch"); val_ds.set_format("torch"); test_ds.set_format("torch")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/492547 [00:00<?, ? examples/s]

Map:   0%|          | 0/56145 [00:00<?, ? examples/s]

Map:   0%|          | 0/52878 [00:00<?, ? examples/s]

In [ ]:
import torch, numpy as np
from sklearn.utils.class_weight import compute_class_weight
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

num_labels = len(label_list)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels, id2label=id2label, label2id=label2id,  low_cpu_mem_usage=True,         # streams weights, lower peak RAM
    torch_dtype=torch.float32
)

# Make sure these are *copies*, not views
df_train = df_train.copy()
df_val   = df_val.copy()
df_test  = df_test.copy()

# Class weights to handle imbalance
y_train = df_train["label"].values
classes = np.arange(num_labels)
weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights = torch.tensor(weights, dtype=torch.float)

class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # ensure weights live with the model and match logits dtype later
        self.class_weights = class_weights.to(self.model.device)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        # remove labels from inputs so the model doesn't compute its own loss
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # make sure weights use the same dtype/device as logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=self.class_weights.to(logits.device).to(logits.dtype))
        loss = loss_fct(
            logits.view(-1, self.model.config.num_labels),
            labels.view(-1)
        )
        return (loss, outputs) if return_outputs else loss


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    prec, rec, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    acc = accuracy_score(labels, preds)
    return {"macro_f1": f1, "accuracy": acc}

args = TrainingArguments(
    output_dir="models/xlmr_percodes",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    weight_decay=0.01,
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    report_to=[]
)

trainer = WeightedTrainer(
    class_weights=class_weights.to(model.device),
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
import inspect, sys
import transformers
from transformers import TrainingArguments

print("transformers version:", transformers.__version__)
print("transformers path:", transformers.__file__)
print("TrainingArguments comes from:", TrainingArguments.__module__)
print("TrainingArguments init signature:\n", inspect.signature(TrainingArguments.__init__))


transformers version: 4.57.1
transformers path: /usr/local/lib/python3.12/dist-packages/transformers/__init__.py
TrainingArguments comes from: transformers.training_args
TrainingArguments init signature:
 (self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: float = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_step

# Portugal specific model, BERTimbau base

In [6]:
import pandas as pd
df = pd.read_csv("cmp_quasisentences_per_codes_with_names.csv")

df


FileNotFoundError: [Errno 2] No such file or directory: 'cmp_quasisentences_per_codes_with_names.csv'

In [ ]:
# Keep only good rows + Portuguese
df = df.dropna(subset=["text","per_code","manifesto_id"])
df = df[df["country_name"].str.lower().eq("Portugal")].reset_index(drop=True)

# normalize per_code and rebuild mappings AFTER filtering
df["per_code"] = df["per_code"].str.lower().str.extract(r'(per\d+)')[0]
df = df[df["per_code"].notna()].reset_index(drop=True)

label_list = sorted(df["per_code"].unique())
label2id = {l:i for i,l in enumerate(label_list)}
id2label = {i:l for l,i in label2id.items()}
df["label"] = df["per_code"].map(label2id)


In [ ]:
df

,manifesto_id,party,party_name,party_abbrev,country_name,date,election_date,language,title,progtype,text,per_code,label


## Portuguese dataset, Portuguese trained model BERTimbau base

In [2]:
import pandas as pd

df_portugal = pd.read_csv("pt_cmp_quasisentences_per_codes.csv")

df_portugal

,manifesto_id,party,party_name,party_abbrev,country_name,date,election_date,language,title,progtype,text,per_code
0,35211_199910,35211,Bloco de Esquerda,BE,Portugal,199910,NaN,portuguese,É tempo de ser exigente,NaN,O Bloco de Esquerda disputa as eleições legisl...,per000
1,35211_199910,35211,Bloco de Esquerda,BE,Portugal,199910,NaN,portuguese,É tempo de ser exigente,NaN,Apresentamo-nos ao país com um Manifesto eleit...,per000
2,35211_199910,35211,Bloco de Esquerda,BE,Portugal,199910,NaN,portuguese,É tempo de ser exigente,NaN,A escolha é propositada.,per000
3,35211_199910,35211,Bloco de Esquerda,BE,Portugal,199910,NaN,portuguese,É tempo de ser exigente,NaN,"Apresenta um ""programa de governo"" quem fixe c...",per000
4,35211_199910,35211,Bloco de Esquerda,BE,Portugal,199910,NaN,portuguese,É tempo de ser exigente,NaN,Como o Bloco não se encontra em nenhuma das du...,per000
...,...,...,...,...,...,...,...,...,...,...,...,...
65813,35720_202201,35720,Chega,NaN,Portugal,202201,NaN,portuguese,Programa eleitoral legislativas 2022,NaN,A Juventude CHEGA defende a atribuição de bene...,per701
65814,35720_202201,35720,Chega,NaN,Portugal,202201,NaN,portuguese,Programa eleitoral legislativas 2022,NaN,A Juventude CHEGA propõe o reajustamento dos p...,per504
65815,35720_202201,35720,Chega,NaN,Portugal,202201,NaN,portuguese,Programa eleitoral legislativas 2022,NaN,A Juventude CHEGA propõe a facilidade de acess...,per504
65816,35720_202201,35720,Chega,NaN,Portugal,202201,NaN,portuguese,Programa eleitoral legislativas 2022,NaN,"XIV. Contra os socialismos, Portugal justo, co...",perH


In [3]:
#Changing date into year. Do not need month, so thats removed.

# Get the index position of the 'date' column
pos = df_portugal.columns.get_loc('date')

# Create a new 'year' column by extracting the first 4 digits
df_portugal.insert(pos, 'year', df_portugal['date'].astype(str).str[:4].astype(int))

# Drop the old 'date' column
df_portugal = df_portugal.drop(columns='date')

df_portugal.head()


,manifesto_id,party,party_name,party_abbrev,country_name,year,election_date,language,title,progtype,text,per_code
0,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,NaN,portuguese,É tempo de ser exigente,NaN,O Bloco de Esquerda disputa as eleições legisl...,per000
1,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,NaN,portuguese,É tempo de ser exigente,NaN,Apresentamo-nos ao país com um Manifesto eleit...,per000
2,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,NaN,portuguese,É tempo de ser exigente,NaN,A escolha é propositada.,per000
3,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,NaN,portuguese,É tempo de ser exigente,NaN,"Apresenta um ""programa de governo"" quem fixe c...",per000
4,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,NaN,portuguese,É tempo de ser exigente,NaN,Como o Bloco não se encontra em nenhuma das du...,per000


In [4]:
# Remove election date and progtype since they are completely missing values.

# Drop only if the columns exist
df_portugal = df_portugal.drop(columns=["election_date", "progtype"])

df_portugal.head()

,manifesto_id,party,party_name,party_abbrev,country_name,year,language,title,text,per_code
0,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,O Bloco de Esquerda disputa as eleições legisl...,per000
1,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,Apresentamo-nos ao país com um Manifesto eleit...,per000
2,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,A escolha é propositada.,per000
3,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,"Apresenta um ""programa de governo"" quem fixe c...",per000
4,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,Como o Bloco não se encontra em nenhuma das du...,per000


In [5]:
df_portugal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65818 entries, 0 to 65817
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   manifesto_id  65818 non-null  object
 1   party         65818 non-null  int64 
 2   party_name    65818 non-null  object
 3   party_abbrev  64592 non-null  object
 4   country_name  65818 non-null  object
 5   year          65818 non-null  int64 
 6   language      65818 non-null  object
 7   title         65818 non-null  object
 8   text          65818 non-null  object
 9   per_code      65818 non-null  object
dtypes: int64(2), object(8)
memory usage: 5.0+ MB


In [6]:
cols = ['party_abbrev']

# Show rows with missing values in any of these columns
missing_rows = df_portugal[df_portugal[cols].isna().any(axis=1)]

# Display results
print(f"Number of rows with missing values: {len(missing_rows)}")
missing_rows

Number of rows with missing values: 1226


,manifesto_id,party,party_name,party_abbrev,country_name,year,language,title,text,per_code
45136,35720_201910,35720,Chega,NaN,Portugal,2019,portuguese,Programa político Chega 2019,A MATRIZ POLÍTICA DO CHEGA!,perH
45137,35720_201910,35720,Chega,NaN,Portugal,2019,portuguese,Programa político Chega 2019,UM PARTIDO LIBERAL E CONSERVADOR,perH
45138,35720_201910,35720,Chega,NaN,Portugal,2019,portuguese,Programa político Chega 2019,"De forma agora mais técnica, o CHEGA é um Part...",per201.1
45139,35720_201910,35720,Chega,NaN,Portugal,2019,portuguese,Programa político Chega 2019,"e da Democracia representativa,",per202.3
45140,35720_201910,35720,Chega,NaN,Portugal,2019,portuguese,Programa político Chega 2019,"no primado do Estado de Direito,",per202.1
...,...,...,...,...,...,...,...,...,...,...
65813,35720_202201,35720,Chega,NaN,Portugal,2022,portuguese,Programa eleitoral legislativas 2022,A Juventude CHEGA defende a atribuição de bene...,per701
65814,35720_202201,35720,Chega,NaN,Portugal,2022,portuguese,Programa eleitoral legislativas 2022,A Juventude CHEGA propõe o reajustamento dos p...,per504
65815,35720_202201,35720,Chega,NaN,Portugal,2022,portuguese,Programa eleitoral legislativas 2022,A Juventude CHEGA propõe a facilidade de acess...,per504
65816,35720_202201,35720,Chega,NaN,Portugal,2022,portuguese,Programa eleitoral legislativas 2022,"XIV. Contra os socialismos, Portugal justo, co...",perH


In [7]:
# Set party_abbrev = "CH" whenever party_name == "Chega"
df_portugal.loc[df_portugal["party_name"].eq("Chega"), "party_abbrev"] = "CH"

df_portugal

,manifesto_id,party,party_name,party_abbrev,country_name,year,language,title,text,per_code
0,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,O Bloco de Esquerda disputa as eleições legisl...,per000
1,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,Apresentamo-nos ao país com um Manifesto eleit...,per000
2,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,A escolha é propositada.,per000
3,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,"Apresenta um ""programa de governo"" quem fixe c...",per000
4,35211_199910,35211,Bloco de Esquerda,BE,Portugal,1999,portuguese,É tempo de ser exigente,Como o Bloco não se encontra em nenhuma das du...,per000
...,...,...,...,...,...,...,...,...,...,...
65813,35720_202201,35720,Chega,CH,Portugal,2022,portuguese,Programa eleitoral legislativas 2022,A Juventude CHEGA defende a atribuição de bene...,per701
65814,35720_202201,35720,Chega,CH,Portugal,2022,portuguese,Programa eleitoral legislativas 2022,A Juventude CHEGA propõe o reajustamento dos p...,per504
65815,35720_202201,35720,Chega,CH,Portugal,2022,portuguese,Programa eleitoral legislativas 2022,A Juventude CHEGA propõe a facilidade de acess...,per504
65816,35720_202201,35720,Chega,CH,Portugal,2022,portuguese,Programa eleitoral legislativas 2022,"XIV. Contra os socialismos, Portugal justo, co...",perH


In [8]:
df_portugal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65818 entries, 0 to 65817
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   manifesto_id  65818 non-null  object
 1   party         65818 non-null  int64 
 2   party_name    65818 non-null  object
 3   party_abbrev  65818 non-null  object
 4   country_name  65818 non-null  object
 5   year          65818 non-null  int64 
 6   language      65818 non-null  object
 7   title         65818 non-null  object
 8   text          65818 non-null  object
 9   per_code      65818 non-null  object
dtypes: int64(2), object(8)
memory usage: 5.0+ MB


No missing values after Chega is filled with abbreviation CH.

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)


In [10]:
df_portugal = df_portugal.dropna(subset=["text", "per_code", "manifesto_id"]).copy()
df_portugal["per_code"] = (
    df_portugal["per_code"]
    .astype(str).str.lower().str.extract(r'(per\d+)')[0]
)
df_portugal = df_portugal[df_portugal["per_code"].notna()].reset_index(drop=True)

# Optional: drop very short boilerplate lines
df_portugal = df_portugal[df_portugal["text"].str.len() > 15].reset_index(drop=True)

# Build label maps (IDs for each perXXX)
label_list = sorted(df_portugal["per_code"].unique())
label2id = {l:i for i,l in enumerate(label_list)}
id2label = {i:l for l,i in label2id.items()}
df_portugal["label"] = df_portugal["per_code"].map(label2id)

len(df_portugal), len(label_list)

(59695, 57)

In [11]:
groups = df_portugal["manifesto_id"]

gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
train_idx, hold_idx = next(gss.split(df_portugal, groups=groups))
df_train = df_portugal.iloc[train_idx].copy()
df_hold  = df_portugal.iloc[hold_idx].copy()

gss2 = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=43)
val_idx, test_idx = next(gss2.split(df_hold, groups=df_hold["manifesto_id"]))
df_val  = df_hold.iloc[val_idx].copy()
df_test = df_hold.iloc[test_idx].copy()

len(df_train), len(df_val), len(df_test)

(49888, 3532, 6275)

In [12]:
model_name = "neuralmind/bert-base-portuguese-cased"  # BERTimbau (cased)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

MAX_LEN = 128  # fits on small GPUs; increase to 256 if you have headroom

def tokenize(batch):
    return tokenizer(
        batch["text"], truncation=True, padding="max_length", max_length=MAX_LEN
    )

train_ds = Dataset.from_pandas(df_train[["text","label"]])
val_ds   = Dataset.from_pandas(df_val[["text","label"]])
test_ds  = Dataset.from_pandas(df_test[["text","label"]])

train_ds = train_ds.map(tokenize, batched=True).remove_columns(["text"]).rename_column("label","labels")
val_ds   = val_ds.map(tokenize, batched=True).remove_columns(["text"]).rename_column("label","labels")
test_ds  = test_ds.map(tokenize, batched=True).remove_columns(["text"]).rename_column("label","labels")

train_ds.set_format("torch"); val_ds.set_format("torch"); test_ds.set_format("torch")


tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/49888 [00:00<?, ? examples/s]

Map:   0%|          | 0/3532 [00:00<?, ? examples/s]

Map:   0%|          | 0/6275 [00:00<?, ? examples/s]

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

num_labels = len(label_list)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels, id2label=id2label, label2id=label2id
)

# Save VRAM if needed
model.gradient_checkpointing_enable()

# Handle class imbalance (many per codes are rare)
y_train = df_train["label"].values
classes = np.arange(num_labels)
weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weights = torch.tensor(weights, dtype=torch.float)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='macro')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'macro_precision': precision,
        'macro_recall': recall,
        'macro_f1': f1,
    }

# Custom Trainer that applies weighted cross-entropy
class WeightedTrainer(Trainer):
    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # store as tensor; we'll move it to the right device inside compute_loss
        if class_weights is not None and not isinstance(class_weights, torch.Tensor):
            class_weights = torch.tensor(class_weights, dtype=torch.float)
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # don't mutate inputs; make a view without labels
        labels = inputs["labels"]
        model_inputs = {k: v for k, v in inputs.items() if k != "labels"}

        outputs = model(**model_inputs)
        logits = outputs.logits if hasattr(outputs, "logits") else outputs["logits"]

        # move weights to the same device as logits
        if self.class_weights is not None:
            weight = self.class_weights.to(logits.device)
            loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        else:
            loss_fct = torch.nn.CrossEntropyLoss()

        loss = loss_fct(
            logits.view(-1, self.model.config.num_labels),
            labels.view(-1)
        )
        return (loss, outputs) if return_outputs else loss

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
use_fp16 = torch.cuda.is_available()  # mixed precision if GPU exists

args = TrainingArguments(
    output_dir="models/bertimbau_percodes",
    learning_rate=2e-5,
    per_device_train_batch_size=16,     # if OOM, try 8 or 4 and add grad_accum
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,      # bump to 2–4 if you reduce batch size
    num_train_epochs=4,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    weight_decay=0.01,
    logging_steps=100,
    fp16=use_fp16,
    report_to=[]
)

trainer = WeightedTrainer(
    class_weights=class_weights.to(model.device),
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Macro Precision,Macro Recall,Macro F1
1,2.151400,2.243834,0.454417,0.302989,0.366779,0.299168
2,1.771100,2.153014,0.473952,0.351859,0.418883,0.331488
3,1.335200,2.186188,0.486410,0.335088,0.443554,0.336582
4,1.092200,2.229183,0.494621,0.340528,0.433875,0.340585


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_

TrainOutput(global_step=12472, training_loss=1.7440074017752891, metrics={'train_runtime': 2066.4776, 'train_samples_per_second': 96.566, 'train_steps_per_second': 6.035, 'total_flos': 1.31325662896128e+16, 'train_loss': 1.7440074017752891, 'epoch': 4.0})

In [15]:
pred = trainer.predict(test_ds)
y_pred = pred.predictions.argmax(axis=-1)
y_true = pred.label_ids

# Unit-level metrics already printed via compute_metrics

# Manifesto-level share correlation
test_raw = df_test.copy()
test_raw["pred_id"] = y_pred
test_raw["pred_code"] = test_raw["pred_id"].map(id2label)

def shares(vec, labels):
    vc = vec.value_counts(normalize=True)
    return np.array([vc.get(lbl, 0.0) for lbl in labels])

rows=[]
for mid, g in test_raw.groupby("manifesto_id"):
    gold = shares(g["per_code"], label_list)
    pred = shares(g["pred_code"], label_list)
    if gold.sum() > 0 and pred.sum() > 0:
        corr = np.corrcoef(gold, pred)[0,1]
        rows.append((mid, corr, len(g)))
import pandas as pd
doc_corr = pd.DataFrame(rows, columns=["manifesto_id","share_corr","n_units"])
doc_corr.describe()


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,share_corr,n_units
count,4.000000,4.000000
mean,0.878497,1568.750000
std,0.150708,639.231505
min,0.664522,1005.000000
25%,0.827074,1049.250000
50%,0.930422,1482.000000
75%,0.981844,2001.500000
max,0.988622,2306.000000


In [16]:
import json, os
os.makedirs("artifacts/bertimbau_pt", exist_ok=True)

with open("artifacts/bertimbau_pt/label_list.json","w") as f: json.dump(label_list, f)
with open("artifacts/bertimbau_pt/label2id.json","w") as f: json.dump(label2id, f)

model.save_pretrained("artifacts/bertimbau_pt/model")
tokenizer.save_pretrained("artifacts/bertimbau_pt/model")


('artifacts/bertimbau_pt/model/tokenizer_config.json',
 'artifacts/bertimbau_pt/model/special_tokens_map.json',
 'artifacts/bertimbau_pt/model/vocab.txt',
 'artifacts/bertimbau_pt/model/added_tokens.json',
 'artifacts/bertimbau_pt/model/tokenizer.json')

### Try with writing sentences

In [17]:
# ==== Mini demo: type a sentence and get perXXX predictions ====

import json, torch, numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

# 1) Where your fine-tuned model + label files live (adjust if needed)
MODEL_DIR = "artifacts/bertimbau_pt/model"           # from your training step
LABEL_LIST_PATH = "artifacts/bertimbau_pt/label_list.json"

# 2) Load tokenizer + model
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR, use_fast=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()

# 3) Load label list (perXXX) and build mappings
try:
    with open(LABEL_LIST_PATH, "r", encoding="utf-8") as f:
        label_list = json.load(f)
    id2label = {i: l for i, l in enumerate(label_list)}
    label2id = {l: i for i, l in enumerate(label_list)}
except FileNotFoundError:
    # Fallback: take labels from model config (if saved there)
    id2label = {int(k): v for k, v in model.config.id2label.items()}
    label_list = [id2label[i] for i in sorted(id2label.keys())]
    label2id = {l: i for i, l in enumerate(label_list)}

# 4) Inference helpers
MAX_LEN = 128  # use the same value you trained with

def predict_sentences(sent_list, topk=5):
    """Return top-k perXXX predictions per sentence."""
    enc = tokenizer(
        sent_list, truncation=True, padding=True, max_length=MAX_LEN, return_tensors="pt"
    )
    enc = {k: v.to(device) for k, v in enc.items()}
    with torch.no_grad():
        logits = model(**enc).logits
        probs = F.softmax(logits, dim=-1).cpu().numpy()
    # Top-k per sentence
    top = np.argsort(-probs, axis=1)[:, :topk]
    out = []
    for i, idxs in enumerate(top):
        pred = [(id2label[j], float(probs[i, j])) for j in idxs]
        out.append(pred)
    return out

def print_predictions(sent_list, topk=5):
    results = predict_sentences(sent_list, topk=topk)
    for s, preds in zip(sent_list, results):
        print("\nTEXT:", s)
        for code, p in preds:
            print(f"  - {code:>7}  prob={p:.3f}")

# 5) >>> TYPE YOUR SENTENCES HERE <<<
sentences = [
    "Ter a ambição de colocar o sistema de saúde português entre os dez melhores do mundo, em 2040, de acordo com os seguintes indicadores internacionais"]
print_predictions(sentences, topk=5)




TEXT: Ter a ambição de colocar o sistema de saúde português entre os dez melhores do mundo, em 2040, de acordo com os seguintes indicadores internacionais
  -  per504  prob=0.647
  -  per505  prob=0.275
  -  per303  prob=0.013
  -  per000  prob=0.012
  -  per503  prob=0.010


### Check implementation with 2024 party programmes

In [21]:
# --- A) IMPORTS ---
import os, re, math, json, glob
from pathlib import Path
from typing import List, Dict, Tuple

import numpy as np
import pandas as pd
from tqdm import tqdm
from pypdf import PdfReader

import spacy
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F


In [20]:
#!pip -q install spacy pypdf pandas numpy tqdm
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 783.7 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [57]:
# --- B) CONFIG ---
# Put your PDFs (one file per programme) here. Suggested naming: PARTY_2024_title.pdf
INPUT_PDF_DIR = "/content"   # change to your folder
OUTPUT_DIR    = "/content/outputs_rile"         # results will be written here
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Your fine-tuned PER-code classifier (Hugging Face path or local folder)
MODEL_NAME_OR_PATH = "/content/artifacts/bertimbau_pt/model"  # e.g., "yourname/per-pt-2024" or a local folder

# Inference settings
BATCH_SIZE = 32
MAX_LEN    = 256          # truncate very long units safely (Portuguese policy sentences are usually shorter)
THRESHOLD  = 0.60         # probability threshold for HARD counts; < threshold -> treat as uncoded
USE_GPU    = torch.cuda.is_available()
DEVICE     = torch.device("cuda" if USE_GPU else "cpu")

# Quasi-sentence splitting heuristics
CONJ_SPLIT = r"\b(e|mas|ou|porém|todavia|contudo|entretanto|no entanto)\b"


In [58]:
# --- C) RILE SETS ---
RIGHT = {
    'per104','per201','per203','per305','per401','per402','per407',
    'per414','per505','per601','per603','per605','per606'
}
LEFT  = {
    'per103','per105','per106','per107','per403','per404','per406',
    'per412','per413','per504','per506','per701','per202'
}


In [59]:
# --- D) PDF EXTRACTION ---
def pdf_to_text(path: str) -> str:
    """Extracts text from a (mostly-text) PDF using pypdf."""
    reader = PdfReader(path)
    parts = []
    for page in reader.pages:
        t = page.extract_text() or ""
        parts.append(t)
    raw = "\n".join(parts)
    return clean_pdf_text(raw)

def clean_pdf_text(t: str) -> str:
    """Fix common OCR/formatting issues: hyphenation splits, extra spaces, weird breaks."""
    # Remove line-break hyphenation: "polí-\ntica" -> "política"
    t = re.sub(r"(\w)-\n(\w)", r"\1\2", t)
    # Normalize newlines and spaces
    t = t.replace("\r", " ")
    t = re.sub(r"[ \t]+", " ", t)
    t = re.sub(r"\n{2,}", "\n\n", t)
    # Strip page headers/footers heuristically (optional, gentle)
    t = re.sub(r"\n?\s*Página \d+\s*\n?", "\n", t, flags=re.IGNORECASE)
    return t.strip()


In [60]:
# --- E) SEGMENTATION (spaCy PT + quasi-sentence rules) ---
_nlp = spacy.load("pt_core_news_lg")

def to_sentences(text: str) -> List[str]:
    """Portuguese sentence segmentation using spaCy."""
    doc = _nlp(text)
    sents = [s.text.strip() for s in doc.sents if s.text.strip()]
    return sents

def split_quasi_sentences(sent: str) -> List[str]:
    """
    Split a sentence into quasi-sentences:
    - first split on ';'
    - then split on coordinating conjunctions (only if it creates a clearly new clause)
    """
    parts = []
    for chunk in re.split(r";", sent):
        chunk = chunk.strip()
        if not chunk:
            continue
        # Light heuristic: split on conjunctions only when surrounded by spaces and followed by lowercase (to avoid names)
        splits = re.split(CONJ_SPLIT, chunk, flags=re.IGNORECASE)
        if len(splits) == 1:
            parts.append(chunk)
        else:
            # Reconstruct: re.split keeps separators if we capture them; we want to cut at them
            # Simpler approach: split on ' e ' etc. conservatively
            subparts = re.split(rf"\s{CONJ_SPLIT}\s", chunk, flags=re.IGNORECASE)
            for sp in subparts:
                s2 = sp.strip(" ,.;:—–-")
                if s2:
                    parts.append(s2)
    # Final tidy
    parts = [p for p in (pp.strip() for pp in parts) if len(p) > 0]
    return parts

def text_to_quasis(text: str) -> List[str]:
    """Full pipeline: text -> sentences -> quasi-sentences -> cleaned units."""
    q = []
    for s in to_sentences(text):
        for unit in split_quasi_sentences(s):
            # Drop very short procedural or noise fragments
            if len(unit) < 20:
                continue
            q.append(unit)
    return q


In [61]:
# --- F) BOILERPLATE FILTER (optional) ---
PROCEDURAL_PATTERNS = [
    r"senhor(a)? presidente", r"senhores(as)? deputados", r"aplausos", r"risos",
    r"sumário", r"índice", r"conteúdo", r"agradecimentos", r"nota editorial"
]

def is_boilerplate(unit: str) -> bool:
    low = unit.lower()
    for pat in PROCEDURAL_PATTERNS:
        if re.search(pat, low):
            return True
    # Drop lines that look like pure headings
    if len(unit.split()) <= 4 and unit.isupper():
        return True
    return False


In [62]:
# --- G) LOAD MODEL ---
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME_OR_PATH).to(DEVICE)
model.eval()

# Try to get label mapping from model config
id2label = getattr(model.config, "id2label", None)
if id2label is None or not isinstance(id2label, dict):
    # Fallback: assume labels 0..num_labels-1 map to 'perXXX' in order (customize if needed)
    id2label = {i: f"per{i:03d}" for i in range(model.config.num_labels)}
label_list = [id2label[i] for i in range(len(id2label))]


In [63]:
from pathlib import Path
import os, json

print("MODEL_NAME_OR_PATH =", MODEL_NAME_OR_PATH)
p = Path(MODEL_NAME_OR_PATH)
print("Exists?", p.exists(), " | Is dir?", p.is_dir())
if p.exists() and p.is_dir():
    print("Files:", sorted(os.listdir(p))[:20])


MODEL_NAME_OR_PATH = /content/artifacts/bertimbau_pt/model
Exists? True  | Is dir? True
Files: ['config.json', 'model.safetensors', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']


In [64]:
# --- G) BATCH PREDICT ---
def predict_per_codes(units: List[str], batch_size=BATCH_SIZE, max_len=MAX_LEN) -> Tuple[List[str], List[float]]:
    """Return (pred_label, pred_prob) per unit."""
    preds, probs = [], []
    for i in tqdm(range(0, len(units), batch_size), desc="Classifying"):
        batch = units[i:i+batch_size]
        enc = tokenizer(batch, padding=True, truncation=True, max_length=max_len, return_tensors="pt").to(DEVICE)
        with torch.no_grad():
            logits = model(**enc).logits
            p = F.softmax(logits, dim=-1)
            conf, idx = torch.max(p, dim=-1)
            preds.extend([label_list[j] for j in idx.tolist()])
            probs.extend(conf.tolist())
    return preds, probs


In [65]:
# --- H) AGGREGATION HELPERS ---
def make_units_df(doc_id: str, party: str, units: List[str], labels: List[str], conf: List[float]) -> pd.DataFrame:
    return pd.DataFrame({
        "doc_id": doc_id,
        "party" : party,
        "unit_id": np.arange(len(units)),
        "text": units,
        "per_pred": labels,
        "prob": conf
    })

def hard_shares(df: pd.DataFrame, threshold: float) -> pd.DataFrame:
    d = df.copy()
    d["per_final"] = np.where(d["prob"] >= threshold, d["per_pred"], "peruncod")
    d = d[d["per_final"] != "peruncod"]
    grp = d.groupby(["doc_id","party","per_final"]).size().rename("n").reset_index()
    tot = grp.groupby(["doc_id","party"])["n"].sum().rename("N").reset_index()
    out = grp.merge(tot, on=["doc_id","party"])
    out["share"] = 100 * out["n"] / out["N"]
    return out  # columns: doc_id, party, per_final, n, N, share

def soft_shares(df: pd.DataFrame) -> pd.DataFrame:
    # Sum probabilities per predicted code (soft counting)
    grp = (df.groupby(["doc_id","party","per_pred"])["prob"].sum()
             .rename("soft_n").reset_index())
    tot = grp.groupby(["doc_id","party"])["soft_n"].sum().rename("soft_N").reset_index()
    out = grp.merge(tot, on=["doc_id","party"])
    out["share"] = 100 * out["soft_n"] / out["soft_N"]
    out = out.rename(columns={"per_pred":"per_final"})
    return out  # columns: doc_id, party, per_final, soft_n, soft_N, share

def rile_from_shares(shares_df: pd.DataFrame) -> float:
    r = shares_df[shares_df["per_final"].isin(RIGHT)]["share"].sum()
    l = shares_df[shares_df["per_final"].isin(LEFT)]["share"].sum()
    return float(r - l)


In [69]:
# --- I) PIPELINE DRIVER ---
def parse_party_and_docid(pdf_path: str) -> Tuple[str, str]:
    """
    Heuristic: filename like 'PSD_2024_programa.pdf' -> party='PSD', doc_id='PSD_2024_programa'
    If your naming is different, customize this.
    """
    name = Path(pdf_path).stem
    m = re.match(r"([A-Za-zÀ-ÿ0-9\-]+)_(\d{4})", name)
    party = name.split("_")[0] if m is None else m.group(1)
    doc_id = name
    return party, doc_id

def process_one_pdf(pdf_path: str) -> Dict[str, any]:
    party, doc_id = parse_party_and_docid(pdf_path)

    # 1) Read & clean
    raw = pdf_to_text(pdf_path)

    # 2) Segment to quasi-sentences
    units = text_to_quasis(raw)
    # Optional: filter boilerplate
    units = [u for u in units if not is_boilerplate(u)]

    if len(units) == 0:
        print(f"[WARN] No usable units for {pdf_path}")
        return None

    # 3) Classify
    labels, conf = predict_per_codes(units)

    units_df = make_units_df(doc_id, party, units, labels, conf)
    units_csv = Path(OUTPUT_DIR) / f"{doc_id}_units.csv"
    units_df.to_csv(units_csv, index=False, encoding="utf-8")

    # 4) Aggregate
    hard = hard_shares(units_df, THRESHOLD)
    soft = soft_shares(units_df)

    # 5) RILE
    rile_hard = (hard.groupby(["doc_id","party"])
                      .apply(rile_from_shares)
                      .reset_index(name="RILE_hard"))
    rile_soft = (soft.groupby(["doc_id","party"])
                      .apply(rile_from_shares)
                      .reset_index(name="RILE_soft"))

    # 6) Save per-programme shares
    hard.to_csv(Path(OUTPUT_DIR)/f"{doc_id}_per_shares_hard.csv", index=False, encoding="utf-8")
    soft.to_csv(Path(OUTPUT_DIR)/f"{doc_id}_per_shares_soft.csv", index=False, encoding="utf-8")

    # Merge RILE variants
    merged = rile_hard.merge(rile_soft, on=["doc_id","party"], how="outer")
    return {
        "doc_id": doc_id,
        "party": party,
        "RILE_hard": float(merged["RILE_hard"].iloc[0]) if "RILE_hard" in merged else np.nan,
        "RILE_soft": float(merged["RILE_soft"].iloc[0]) if "RILE_soft" in merged else np.nan,
        "n_units": len(units)
    }

def run_all():
    pdfs = sorted(glob.glob(str(Path(INPUT_PDF_DIR) / "*.pdf")))
    results = []
    for pdf in pdfs:
        print(f"\n=== Processing: {pdf} ===")
        res = process_one_pdf(pdf)
        if res is not None:
            results.append(res)
    if results:
        df = pd.DataFrame(results)
        df.sort_values(["party","doc_id"], inplace=True)
        out_csv = Path(OUTPUT_DIR) / "RILE_results.csv"
        df.to_csv(out_csv, index=False, encoding="utf-8")
        print("\nWrote:", out_csv)
        display(df)
    else:
        print("No results. Check your input folder and filenames.")

# RUN
run_all()



=== Processing: /content/CH_2024.pdf ===


Classifying: 100%|██████████| 125/125 [00:13<00:00,  9.25it/s]
/tmp/ipython-input-3701599760.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rile_from_shares)
/tmp/ipython-input-3701599760.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rile_from_shares)



=== Processing: /content/PSD_2022.pdf ===


Classifying: 100%|██████████| 158/158 [00:14<00:00, 10.72it/s]
/tmp/ipython-input-3701599760.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rile_from_shares)
/tmp/ipython-input-3701599760.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rile_from_shares)



=== Processing: /content/PSD_2024.pdf ===


...


=== Processing: /content/PS_2024.pdf ===


Classifying: 100%|██████████| 126/126 [00:12<00:00,  9.84it/s]



Wrote: /content/outputs_rile/RILE_results.csv


/tmp/ipython-input-3701599760.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rile_from_shares)
/tmp/ipython-input-3701599760.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(rile_from_shares)


,doc_id,party,RILE_hard,RILE_soft,n_units
0,CH_2024,CH,4.682109,3.043109,3985
3,PS_2024,PS,-8.960915,-8.811035,4031
1,PSD_2022,PSD,-3.407290,-3.103034,5031
2,PSD_2024,PSD,-4.229323,-4.361489,4771
